<a href="https://colab.research.google.com/github/NaveedHakim3899/HateSpeechClassificationUsingBert/blob/main/BERT_TDMR_Final_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "tensorflow-text==2.9.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
#bert as it is without any fine tuning or anything
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [ ]:
#Defining an input layer of text in data in a neural network. 
#tf.keras.layers.Input: This is a function in TensorFlow that creates an input layer for a neural network model.

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input) 
outputs= bert_encoder(preprocessed_text)

In [ ]:
#neuralnetwork layers 
#drop out helps in avoiding overfitting
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
#Sigmoid for binary output
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l) 
#passing whatever we got from previous layer to this layer in a function style 
model=tf.keras.Model(inputs=[text_input], outputs =[l]) #The architechure supporting multiple input and output layers


METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
       tf.keras.metrics.Recall(name='F1')
]
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FinalData.csv")
print(len(df))
df['Truth'].value_counts()
#Is it valid? Since Not using validation set Separately 



X_train, X_test, y_train, y_test = train_test_split(df['Content'],df['Truth'], stratify=df['Truth'])
#it can be cleaned more, should I?

298


In [ ]:
#model.fit(X_train, y_train, epochs=100)
#Fitting the architechure on this 
model.fit(df['Content'], df['Truth'], epochs =10)

Epoch 1/10
10/10 [==============================] - 190s 18s/step - loss: 0.7752 - accuracy: 0.5101 - precision: 0.5244 - recall: 0.2867 - F1: 0.2867
Epoch 2/10
10/10 [==============================] - 174s 17s/step - loss: 0.7481 - accuracy: 0.4933 - precision: 0.4982 - recall: 0.9133 - F1: 0.9133
Epoch 3/10
10/10 [==============================] - 172s 17s/step - loss: 0.7367 - accuracy: 0.5067 - precision: 0.5556 - recall: 0.1000 - F1: 0.1000
Epoch 4/10
10/10 [==============================] - 173s 17s/step - loss: 0.7060 - accuracy: 0.5336 - precision: 0.5288 - recall: 0.6733 - F1: 0.6733
Epoch 5/10
10/10 [==============================] - 177s 18s/step - loss: 0.7207 - accuracy: 0.4732 - precision: 0.4822 - recall: 0.6333 - F1: 0.6333
Epoch 6/10
10/10 [==============================] - 172s 17s/step - loss: 0.7282 - accuracy: 0.4832 - precision: 0.4825 - recall: 0.3667 - F1: 0.3667
Epoch 7/10
10/10 [==============================] - 169s 17s/step - loss: 0.6967 - accuracy: 0.5403 

In [ ]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 43s 12s/step - loss: 0.7002 - accuracy: 0.5200 - precision: 0.5278 - recall: 0.5000 - F1: 0.5000


[0.7001511454582214, 0.5199999809265137, 0.5277777910232544, 0.5, 0.5]